In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import pdir as pr
import pandas as pd
import os
from tqdm import tqdm, tnrange, tqdm_notebook

DF = pd.DataFrame
arr = np.array

# 读取数据集

In [9]:
dirPath2 = "E:\\Code\\_largeData\\Github--Open-Course-Learning--A04\\Project\\multiclass classification\\data preprocessed\\wordvec_6B_50d - TA version" 


trainSet = np.loadtxt(dirPath2 + '\\train.csv', delimiter=",")
validateSet = np.loadtxt(dirPath2 + '\\validate.csv', delimiter=",")
testSet = np.loadtxt(dirPath2 + '\\test.csv', delimiter=",")

dirPath = "E:\\Code\\_largeData\\Github--Open-Course-Learning--A04\\Project\\multiclass classification\\data preprocessed\\tf-idf"
trainSetLabel = np.loadtxt(dirPath + '\\train_label.csv', delimiter=",")
validateSetLabel = np.loadtxt(dirPath + '\\validate_label.csv', delimiter=",")

trainSet.shape, validateSet.shape, testSet.shape
trainSetLabel.shape, validateSetLabel.shape

((43766, 50), (18756, 50), (8671, 50))

((43766,), (18756,))

# KNN

In [10]:
def knn_classify(dataSet, labels, k, inputVector):
    '''使用knn对输入的向量进行分类，使用欧式距离'''
    #得到输入向量与数据集的差值的平方
    diffMat = (dataSet - inputVector)**2
    #计算每一行上元素的和
    distancesSquare = diffMat.sum(axis=1)
    #可以不用计算距离，用距离的平方也可以，这样效率更高
    #print(distancesSquare)
    #得到 K 个近邻的下标
    kNeighborsIndex  = distancesSquare.argpartition(k-1)[0:k]
    
    #计算 K 个近邻的距离倒数
    kDatas = 1/distancesSquare[kNeighborsIndex]
    #得到 对应的 K 个标签
    kLabels = labels[kNeighborsIndex]
    
    predictLabel = np.NAN
    maxDis = -np.inf
    for i in np.unique(kLabels):
        dis = np.sum(kDatas[kLabels==i]) 
        #print(dis)
        if dis > maxDis:
            maxDis = dis
            predictLabel = i
    #返回分类结果
    return predictLabel

##############测试程序#######################
knn_classify(arr([[1,1,0,0],
              [0,1,1,1],
              [1,0,0,1]]), arr([1, 0, 1]), 3, arr([0,1,1,0]))
##############测试程序#######################

0

# 计算平均准确率

In [11]:
def calcAvg(predict, actual):
    all_labels = list(set(actual))
    tot = 0.0
    for i in all_labels:
        counter = Counter(predict[actual == i] == i)
        hit_num, miss_num = counter[True], counter[False]
        tot += hit_num / (hit_num + miss_num)
    return tot/len(all_labels)

calcAvg(arr([1,2,3]), arr([3,2,1]))

0.3333333333333333

In [12]:
trainSet[0,:]

array([ 0.23078921,  0.27323677, -0.02954577, -0.04853166,  0.28180856,
        0.0592143 , -0.42988769, -0.02372992, -0.03119839, -0.14007146,
       -0.15106738, -0.12000525, -0.14026137, -0.07444043,  0.27119987,
        0.13343253,  0.10473174,  0.05121908, -0.37115628, -0.23900846,
        0.06559   ,  0.09519941,  0.09231085,  0.06028896,  0.04956119,
       -1.22946585, -0.21750375,  0.17548555,  0.24603385, -0.17863073,
        2.47236468,  0.15369629, -0.12115178, -0.08087972,  0.0796592 ,
       -0.05632091, -0.04942061,  0.03542139,  0.02938966, -0.07193589,
       -0.18563327,  0.12049308,  0.10473326, -0.00497364,  0.01172083,
        0.14822382, -0.14478865, -0.09108413, -0.06119572, -0.0590018 ])

In [13]:
def run_knn_classify(dataSet, k):
    ansLabel = []
    for i in tnrange(len(dataSet), leave=False):
        ansLabel.append(knn_classify(trainSet, trainSetLabel, k, dataSet[i]))
    return arr(ansLabel)

def getTestAvg(k):
    ansLabel = run_knn_classify(validateSet, k)
    return calcAvg(ansLabel, validateSetLabel)


In [14]:
testRange = np.arange(1, 15)
testRange

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [ ]:
avgs = []
for k in tnrange(1, len(testRange)+1):
    avgs.append(getTestAvg(k))

In [ ]:
avgs

In [ ]:
bestK = testRange[np.argmax(avgs)]
bestK, np.max(avgs) 

In [ ]:
def show_fig():
    fig,ax = plt.subplots() 
    fig.set_size_inches(10, 4)
    plt.plot(testRange, avgs, 'g')
    plt.xlabel('K')
    plt.ylabel('avg(%)')
    plt.title("avg versus K with KNN")
    plt.show()
    plt.close()
show_fig()

In [ ]:
ansLabel = run_knn_classify(testSet, bestK)

In [ ]:
ans = DF(ansLabel).replace(0, "LOW").replace(1, "MID").replace(2, "HIG")
ans.to_csv('.\\rank\\47_v1.csv', index=False, header=False)

Counter(ans[0])